В statsmodels python последняя стабильная версия 0.12. Я рассчитываю:

In [ ]:
import statsmodels.tsa.arima_model as stats

time_series = [2, 3.0, 5, 7, 9, 11, 13, 17, 19]
steps = 4
alpha = 0.05

model = stats.ARIMA(time_series, order=(0, 1, 0))
model_fit = model.fit(disp=0)

forecast, _, intervals = model_fit.forecast(steps=steps, exog=None, alpha=alpha)

Что приводит к
forecast = [21.125, 23.25, 25.375, 27.5]
intervals = [[19.5950036, 22.6549964 ], [21.08625835, 25.41374165], [22.72496851, 28.02503149], [24.44000721, 30.55999279]]

И «Предупреждение о будущем», в котором говорится:
FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

В новой версии, как указано в Future Warning, я вычисляю:

In [ ]:
import statsmodels.tsa.arima.model as stats

time_series = [2, 3.0, 5, 7, 9, 11, 13, 17, 19]
steps = 4
alpha = 0.05

model = stats.ARIMA(time_series, order=(0, 1, 0))
model_fit = model.fit()

forecast = model_fit.get_forecast(steps=steps)
forecasts_and_intervals = forecast.summary_frame(alpha=alpha)

Что дает разные результаты:
forecasts_and_intervals =
y  mean   mean_se  mean_ci_lower  mean_ci_upper
0  19.0  2.263842      14.562951      23.437049
1  19.0  3.201556      12.725066      25.274934
2  19.0  3.921089      11.314806      26.685194
3  19.0  4.527684      10.125903      27.874097

Хотелось бы получить те же результаты, что и раньше. Правильно ли я использую новый интерфейс?

# Ответ

Разница заключается в том, включают ли модели «постоянный» член или нет. Для первого случая, т.е. более старого statsmodels.tsa.arima_model.ARIMA, он автоматически включает постоянный член (и нет возможности включить / выключить). Если у вас есть различие, оно также включает его, но делает это в разностном домене (в противном случае оно все равно было бы удалено). Итак, вот его модель ARIMA (0, 1, 0):

y_t - y_{t-1} = c + e_t

Что является «случайным блужданием со сносом».

Для нового statsmodels.tsa.arima.model.ARIMA, как указано в документации, которую вы связали, не включается какой-либо термин тренда (включая константу, то есть c), когда используется разность, что имеет место для вас. Итак, вот его модель ARIMA (0, 1, 0):

y_t - y_{t-1} = e_t

Что является «случайным блужданием» и, как мы знаем, прогнозы на его основе соответствуют наивным прогнозам, то есть повторению последнего значения (19 в вашем случае).

Тогда что делать, чтобы новый заработал?

Он включает параметр под названием trend, который вы можете указать, чтобы получить такое же поведение. Поскольку вы используете разность (d = 1), передача trend="t" должна дать ту же модель, что и старая. ("t" означает линейный тренд, но поскольку d = 1, он будет уменьшаться до константы в разностной области):

In [ ]:
import statsmodels.tsa.arima.model as stats

time_series = [2, 3.0, 5, 7, 9, 11, 13, 17, 19]
steps = 4
alpha = 0.05

model = stats.ARIMA(time_series, order=(0, 1, 0), trend="t")   # only change is here!
model_fit = model.fit()

forecast = model_fit.get_forecast(steps=steps)
forecasts_and_intervals = forecast.summary_frame(alpha=alpha)

И вот что я получаю за forecasts_and_intervals:

y       mean   mean_se  mean_ci_lower  mean_ci_upper
0  21.124995  0.780622      19.595004      22.654986
1  23.249990  1.103966      21.086256      25.413724
2  25.374985  1.352077      22.724962      28.025008
3  27.499980  1.561244      24.439997      30.559963